In [7]:
import numpy as np
from scipy.stats import qmc
import ipywidgets as widgets
from IPython.display import display
import os
import random

In [8]:
# Define the relative path to the output file within the repo
relativePath = os.path.join("..", "random_number_sample", "generated_numbers_2D.txt")

In [9]:
# Define the function to generate quasirandom numbers
def generateQuasiRandom(fileName, quasirandomType, numDimensions, numNumbers, rangeMin, rangeMax, seed):
    # Define range scaling function
    def scaleRange(samples, rangeMin, rangeMax):
        return samples * (rangeMax - rangeMin) + rangeMin
    if seed == -1:  # Check if seed is "-1"
        seed = random.randint(0, 4294967295) #2^32-1
        print(f"Randomly generated seed: {seed}")

    # Generate quasirandom numbers based on selected type
    if quasirandomType == "Halton":
        sampler = qmc.Halton(d=numDimensions, seed=seed) #, scramble=True
        samples = sampler.random(numNumbers)
    # sobol require atleast 2 dim 1 is posible (normals use for power of 2 ex. 128 256 512 1024)
    elif quasirandomType == "Sobol": 
        sampler = qmc.Sobol(d=numDimensions, seed=seed) #, scramble=True
        samples = sampler.random_base2(numNumbers)
    elif quasirandomType == "LHS":
        sampler = qmc.LatinHypercube(d=numDimensions, seed=seed)
        samples = sampler.random(numNumbers)
    # for safety
    else:
        print("Unsupported quasirandom type.")
        return
    
    # Scale samples to the specified range
    scaledSamples = scaleRange(samples, rangeMin, rangeMax)
    
    # Save to file
    np.savetxt(fileName, scaledSamples, delimiter=',', comments='')
    print(f"from {seed} use {quasirandomType} to generate {numNumbers} quasirandom numbers between {rangeMin} and {rangeMax} written to written to {fileName}.")

In [10]:
# Define a common style for label width
labelStyle = {'description_width': 'initial'}

# Create widgets for input
fileNameWidget = widgets.Text(description="File Name:", value=relativePath, layout=widgets.Layout(width="500px"), style=labelStyle)
quasiTypeWidget = widgets.Dropdown(options=["Halton", "Sobol", "LHS"], description="Type:", style=labelStyle)
numDimensionsWidget = widgets.IntText(description="Dimensions:", value=2, style=labelStyle)
numNumbersWidget = widgets.IntText(description="Number(sobol:2 power this number):", value=100000, style=labelStyle)
rangeMinWidget = widgets.IntText(description="Min Range:", value=0, style=labelStyle)
rangeMaxWidget = widgets.IntText(description="Max Range:", value=20000, style=labelStyle)
seedWidget = widgets.IntText(description="Seed (-1=random):", value=-1, style=labelStyle)

In [11]:
# Define function for button click
def onButtonClick(b):
    fileName = fileNameWidget.value
    quasirandomType = quasiTypeWidget.value
    numDimensions = numDimensionsWidget.value
    numNumbers = numNumbersWidget.value
    rangeMin = rangeMinWidget.value
    rangeMax = rangeMaxWidget.value
    seed = seedWidget.value

    # Validate that numNumbers is greater than 0 
    if numNumbers < 0 or numDimensions < 0:
        if numNumbers < 0:
            print("\nError: 'Numbers' must be greater than 0.\n")
            return
        if numDimensions < 0:
            print("\nError: 'numDimensions' must be greater than 0.\n")
            return
    else:
        generateQuasiRandom(fileName, quasirandomType, numDimensions, numNumbers, rangeMin, rangeMax, seed)

In [12]:
# Create button
generateButton = widgets.Button(description="Generate Numbers")
generateButton.on_click(onButtonClick)

# Display the widgets
display(fileNameWidget, quasiTypeWidget, numDimensionsWidget, numNumbersWidget, rangeMinWidget, rangeMaxWidget, seedWidget, generateButton)

Text(value='..\\random_number_sample\\generated_numbers_2D.txt', description='File Name:', layout=Layout(width…

Dropdown(description='Type:', options=('Halton', 'Sobol', 'LHS'), style=DescriptionStyle(description_width='in…

IntText(value=2, description='Dimensions:', style=DescriptionStyle(description_width='initial'))

IntText(value=100000, description='Number(sobol:2 power this number):', style=DescriptionStyle(description_wid…

IntText(value=0, description='Min Range:', style=DescriptionStyle(description_width='initial'))

IntText(value=20000, description='Max Range:', style=DescriptionStyle(description_width='initial'))

IntText(value=-1, description='Seed (-1=random):', style=DescriptionStyle(description_width='initial'))

Button(description='Generate Numbers', style=ButtonStyle())